**Data Cleaning/Preprocessing**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from sklearn.linear_model import LinearRegression, LogisticRegression
import duckdb

These four dataframes we are using are split from the same dataframe but we will later combine them to make querying simpler

In [2]:
medication_df = pd.read_csv("Medication_Data_Kaggle_V1.csv")
socio_eco_df = pd.read_csv("Socio_Economics_Data_Kaggle_V1.csv")
patient_data_training_df = pd.read_csv("Patient_Data_Training_Kaggle_V1.csv")
patient_data_testing_df = pd.read_csv("Patient_Data_Testing_Features.csv")

**Renaming column names to include no spaces for simpler selection**

In [3]:
def rename_column(colname):
  return colname.replace(' ', '_')

medication_df = \
medication_df.rename(mapper = rename_column, axis = 1)

socio_eco_df = \
socio_eco_df.rename(mapper = rename_column, axis = 1)

patient_data_training_df = \
patient_data_training_df.rename(mapper = rename_column, axis = 1)

patient_data_testing_df = \
patient_data_testing_df.rename(mapper = rename_column, axis = 1)



**Combining patient_data dataframes:** The dataset we are using has a training set and a test set that had been split so we will recombine these dataframes and have one patient_data_df

In [4]:
patient_data_df = pd.concat\
([patient_data_training_df, patient_data_testing_df], \
 axis = 0, ignore_index = True)

patient_data_df

,ID,Year,Age,Race,Sex,State,Zip_Code,MSA,Enrollment_Type_Categorized,Enrollment_Months,...,#_Part_B_E&M,#_Part_B_Drugs,#_Part_B_Ambulance,#_Dialysis,#_Part-B_Dialysis,#_Rx_Claims,#_DME,#_Miscellaneous,TotalCost_Y_Actual,TotalCost_Y_Expected
0,68802021,2021,68,White,Male,Michigan,48176,11460,10,12.0,...,2.0,1,0,0,0.0,16,6,0,41679.13,41679.13
1,3422021,2021,60,Black,Male,Michigan,49224,12980,20,12.0,...,0.0,3,0,0,0.0,465,0,0,2045.06,2045.06
2,7892017,2017,43,White,Male,Michigan,49016,12980,20,12.0,...,0.0,0,0,0,0.0,100,0,0,419.49,419.49
3,7892020,2020,46,White,Male,Michigan,49016,12980,20,12.0,...,0.0,0,0,0,0.0,182,0,0,1540.12,1540.12
4,42792020,2020,84,Black,Female,Michigan,49224,12980,10,12.0,...,1.0,0,0,0,0.0,28,0,0,8412.82,8412.82
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20407,85562017,2017,31,Black,Female,Michigan,48503,22420,20,12.0,...,45.0,0,10,0,0.0,139,0,0,NaN,NaN
20408,37692018,2018,69,Black,Female,Michigan,48911,29620,10,12.0,...,33.0,0,7,0,0.0,112,10,0,NaN,NaN
20409,64692018,2018,55,Black,Female,Michigan,48617,99022,20,12.0,...,42.0,0,13,0,0.0,159,35,0,NaN,NaN
20410,62092018,2018,34,White,Male,Michigan,48910,29620,20,12.0,...,97.0,4,7,0,0.0,131,9,0,NaN,NaN


The patient_data_df dataframe has one more row than the other two dataframes medication_df and socio_eco_df and after checking for duplicates, we see that there is a duplicated ID. After removing this duplicated ID, patient_data_df now has 20411 rows, same as the other dataframes. These dataframes are now ready to combine in the next step. Note: We could have combined even with uneven row lengths but it was important that the duplicated ID was removed because each value in ID should be unique, representing a unique patient.

In [5]:
print(np.shape(patient_data_df)) #old shape of patient_data_df
len(patient_data_df['ID'].unique())

patient_data_df = patient_data_df.drop_duplicates(subset = 'ID')

print(np.shape(patient_data_df)) #new shape with duplicated row removed

(20412, 44)
(20411, 44)


1. Combining Dataframes

We use a left join to join the three dataframe together. After the dataframes are joined together, there are some duplicate columns as each of the dataframes have the same 'ID' and 'Year' columns so we removed duplicate columns. This new combined dataframe is called diabetes_df

In [8]:
diabetes_df = duckdb.sql("SELECT * FROM patient_data_df \
LEFT JOIN medication_df \
ON patient_data_df.ID = medication_df.ID \
LEFT JOIN socio_eco_df \
ON patient_data_df.ID = socio_eco_df.ID").df()

diabetes_df = diabetes_df.drop(['ID_2', 'ID_3', 'Year_2', 
                                'Year_3'], 
                               axis = 1)


The dataframe we used for the final report is made from diabetes_1.csv which is a slightly modified version of the file below, the only modification was manually deleting the index column on excel instead of pandas

In [ ]:
diabetes_df.to_csv('diabetes.csv')